## Preprocessing and cleaning the dataset

The following code preprocesses and cleans the dataset.  We read the data from the file, extract the useful data that we wish to use, create our datasets and fill in any NaNs using the mean.

In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import re
import unicodedata

path_load = 'pp-complete.csv'
path_train = 'train.csv'
path_val = 'val.csv'
path_test = 'test.csv'

In [2]:
def convert_num(s):
    if str.isnumeric(s):
        # check no 'vulgar fractions'
        # https://regexr.com/3p8nd
        if re.search(r'[\u2150-\u215F\u00BC-\u00BE\u2189]', s) == None:
            return float(s)
        else:
            return convert_num(s[:-1])
    else:
        return np.NaN


def is_london(s):
    return int("LONDON" in s.upper())


cols = [1, 2, 4, 7, 11]
names = ['price', 'date', 'prop_type', 'lease', 'in_london']
cat_type = CategoricalDtype(categories=list('DSTFO'), ordered=False)
types = {'prop_type': cat_type}
converters = {'lease': convert_num, 'in_london': is_london}
reader = pd.read_csv(path_load, usecols=cols, chunksize=10 ** 7, header=None, names=names,
                     parse_dates=[1], dtype=types, converters=converters)


def process_chunk(chunk):
    # drop rows with too many nan's
    chunk = chunk.dropna(subset=['price', 'date'])
    chunk = chunk.dropna(thresh=3)

    # get dummies (one_hot vectors)
    chunk = pd.get_dummies(chunk)

    # split into train, val, test sets and drop the 'date' column
    chunk['date'] = chunk['date'].dt.year
    df_train = chunk[chunk['date'] < 2014].drop('date', axis=1)
    df_val = chunk[chunk['date'] == 2014].drop('date', axis=1)
    df_test = chunk[chunk['date'] == 2015].drop('date', axis=1)

    # fill nan's on train and val set but fill nan's on test set from 2014 data in second step
    df_train.lease = df_train.lease.fillna(
        df_train.lease.mean()).astype(np.int32)
    df_val.lease = df_val.lease.fillna(df_val.lease.mean()).astype(np.int32)

    # save data
    df_train.to_csv(path_train, mode='a', header=False, index=False)
    df_val.to_csv(path_val, mode='a', header=False, index=False)
    df_test.to_csv(path_test, mode='a', header=False, index=False)


def clean_test_data():
    df_val_lease = pd.read_csv(path_val, header=None, usecols=[
                               1], index_col=False, squeeze=True)
    names_test = ['price', 'lease', 'in_london', 'D', 'S', 'T', 'F', 'O']
    df_test = pd.read_csv(path_test, header=None,
                          index_col=False, names=names_test)

    df_test['lease'] = df_test['lease'].fillna(
        df_val_lease.mean()).astype(np.int32)

    df_test.to_csv(path_test, mode='w', header=False, index=False)

In [3]:
for chunk in reader:
    process_chunk(chunk)

In [4]:
clean_test_data()

In [5]:
print("Now run in bash: shuf train.csv > train_shuf.csv")

Now run in bash: shuf train.csv > train_shuf.csv


We now run the model in [model.ipynb](model.ipynb)